# Dependency code

In [ ]:
import imp
import os
import sys
import numpy as np
import glob
import cmdbench

fp, pathname, description = imp.find_module('benchmark', ['../lib'])
benchmark = imp.load_module('benchmark', fp, pathname, description)

# Summarize numpy array if it has more than 10 elements
np.set_printoptions(threshold=10)

# Software versions

In [ ]:
!skesa --version

# Benchmarking

## Input data and constants

In [ ]:
input_dir = '../data/input-files/reads'
input_files_1 = [os.path.basename(f) for f in glob.glob(f'{input_dir}/*_1.fastq.gz')]
input_samples = [f.replace('_1.fastq.gz','') for f in input_files_1]
input_samples.sort()
print(input_samples)

skesa_out = 'skesa-output'

nproc = 32

sample_sizes = [1,10,20,30,40,50,60,70,80]
#sample_sizes = [1,5,10]

benchmark.create_folder_if_doesnt_exist(skesa_out)

## Benchmark functions

In [ ]:
def reset_func():
    benchmark.clean_if_exists(skesa_out)
        
def sampling_func(sample_size):
    samples = input_samples[:sample_size]
    return samples

## Benchmark

In [ ]:
assembly_command = {
    "command": f"skesa --fastq {input_dir}/%_1.fastq.gz,{input_dir}/%_2.fastq.gz --cores 1 --contigs_out {skesa_out}/%.fasta",
    "parallel_args": f"-j {nproc} -I%"
}

# active_output_print: prints stdout and stderr on every iteration
multibench_results, debug_str = benchmark.multibench.multi_cmdbench({
    "assembly": [assembly_command]
    },
    reset_func = reset_func, iterations = 1, sampling_func = sampling_func, sample_sizes = sample_sizes, 
    benchmark_list_to_results = benchmark.benchmark_list_to_results, active_output_print = True
)

# Save and reload results

In [ ]:
save_path = "multibench_results.txt"

samples_per_sample_size = []
for sample_size in sample_sizes:
        samples_per_sample_size.append(input_samples[:sample_size])

benchmark.multibench.save_multibench_results(multibench_results, samples_per_sample_size, save_path)

multibench_results, samples_per_sample_size = benchmark.multibench.read_multibench_results(save_path)
print(samples_per_sample_size)

# Plot

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np
from pylab import rcParams
rcParams['figure.figsize'] = 15, 3

In [ ]:
# Indexing Plots
benchmark.multibench.plot_resources(multibench_results, sample_sizes, "assembly")